In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
data = pd.read_excel('./data/clean/data_all_clean.xlsx')
data.drop(columns=['Unnamed: 0'], inplace=True)

In [14]:
data.drop(columns=['Unnamed: 0.1','Unnamed: 0'], inplace=True)

# PLAN 

1. data_1: Original dataset. 
    1. data_1_imputed: MICE(data_1)
    2. data_1_imputed_gen: tGAN(data_1_imputed)
2. data_2: Drop only the rows where mathematics is missing.
    1. data_2_imputed: MICE(data_2)
    2. data_2_imputed_gen: tGAN(data_2_imputed)
2. data_3: Drop all the missing rows
    1. data_3_gen: tGAN(data_3)


In [35]:
data.columns

Index(['Unnamed: 0', 'ID', 'Szervezet_neve', 'program_code', 'Felvétel_féléve',
       'ues', 'extra_points', 'final_status', 'Oklevél_eredménye_számmal',
       'financing_source', 're_enrolling', 'foreign_lang', 'hun_lang_lit',
       'math', 'history', 'elective_subj', 'Hallgató_ID', 'foreign_lang_hs',
       'hun_lit_hs', 'hun_lang_hs', 'math_hs', 'history_hs', 'science_hs',
       'foreign_lang_score', 'gender', 'years_between', 'surplus_score',
       'place_of_hs', 'competition', 'ues_calc_method'],
      dtype='object')

In [15]:
from collections import Counter

In [20]:
data.competition.isna().sum()

21633

In [21]:
len(data)

22125

In [36]:
data.drop(columns=['Unnamed: 0','Felvétel_féléve', 'Hallgató_ID', 'ID', 'competition', 'Oklevél_eredménye_számmal', 'Szervezet_neve'], inplace=True)

In [37]:
data.head()

,program_code,ues,extra_points,final_status,financing_source,re_enrolling,foreign_lang,hun_lang_lit,math,history,...,hun_lang_hs,math_hs,history_hs,science_hs,foreign_lang_score,gender,years_between,surplus_score,place_of_hs,ues_calc_method
0,prog_1,417.00,85.00,dropout,state_funded,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Male,NaN,NaN,city_w_county_rights,doubling
1,prog_1,410.00,78.00,graduate,state_funded,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Male,NaN,NaN,city_w_county_rights,doubling
2,prog_2,467.75,93.75,dropout,self_funded,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Male,NaN,NaN,city_w_county_rights,doubling
3,prog_2,459.00,85.00,graduate,state_funded,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Male,NaN,NaN,city_w_county_rights,doubling
4,prog_3,387.00,35.00,dropout,state_funded,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Male,NaN,NaN,city_w_county_rights,doubling


In [38]:
data_3 = data.dropna()

In [39]:
len(data_3)

5239

In [40]:
data_3.dtypes

program_code           object
ues                   float64
extra_points          float64
final_status           object
financing_source       object
re_enrolling             bool
foreign_lang          float64
hun_lang_lit          float64
math                  float64
history               float64
elective_subj         float64
foreign_lang_hs       float64
hun_lit_hs            float64
hun_lang_hs           float64
math_hs               float64
history_hs            float64
science_hs            float64
foreign_lang_score    float64
gender                 object
years_between         float64
surplus_score         float64
place_of_hs            object
ues_calc_method        object
dtype: object

In [17]:
cont_columns = [2,3,4,5,9,10,11,12,13,14,15,16,17,18,19,20,21,22]

In [19]:
data_3['Szervezet_neve'] = data_3.Szervezet_neve.astype(str)
data_3['Képzés'] = data_3.Képzés.astype(str)
data_3['ks_helye'] = data_3.ks_helye.astype(str)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

In [28]:
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit([[1, 2], [3, 6], [4, 8], [np.nan, 3], [7, np.nan]])  

IterativeImputer(add_indicator=False, estimator=None,
                 imputation_order='ascending', initial_strategy='mean',
                 max_iter=10, max_value=None, min_value=None,
                 missing_values=nan, n_nearest_features=None, random_state=0,
                 sample_posterior=False, tol=0.001, verbose=0)

In [29]:
X_test = [[np.nan, 2], [6, np.nan], [np.nan, 6]]
# the model learns that the second feature is double the first
print(np.round(imp.transform(X_test)))

[[ 1.  2.]
 [ 6. 12.]
 [ 3.  6.]]


# <hr>

In [39]:
data = pd.read_excel('./data/clean/data_all_clean.xlsx', dtype={'Képzés': 'category', 'ks_helye': 'category'})

In [44]:
data = pd.read_excel('./data/clean/data_all_clean.xlsx')

In [45]:
# data.drop(columns=['Unnamed: 0', 'Szervezet_neve', 'Felvétel_féléve', 'Hallgató_ID', 'ID', 'Oklevél_eredménye_számmal', 
#                    'felvi_osszes_skalazott', 'Hozott/Tanulmányi_pontszám', 'Szerzett/Érettségi_pontsz'], 
#           inplace=True)

In [46]:
data.drop(columns=['Unnamed: 0','Felvétel_féléve', 'Hallgató_ID', 'ID', 'competition', 'Oklevél_eredménye_számmal', 'Szervezet_neve'], inplace=True)

In [48]:
data = data[data['math']>0]

In [49]:
len(data)

7490

In [44]:
data.nem_ferfi = data.nem_ferfi.astype(bool)

In [26]:
transformed_data = pd.get_dummies(data, dummy_na=False)

Impute missing data

In [112]:
# transformed_data.allamilag_tamogatott = transformed_data.allamilag_tamogatott.astype(int)
# transformed_data.re_enrolled = transformed_data.re_enrolled.astype(int)
# transformed_data.nem_ferfi = transformed_data.nem_ferfi.astype(int)

In [30]:
# from fancyimpute import MICE

In [31]:
# exp_vars = [col for col in transformed_data.columns if col != 'Statusz_vegzett']
# X, y = transformed_data.loc[:,exp_vars], transformed_data.loc[:,'Statusz_vegzett']

In [32]:
# columns = X.columns
# imputed_X = pd.DataFrame(MICE(verbose=False, init_fill_method='median', n_imputations=200).complete(X.values))
# imputed_X.columns = columns

Drop missing rows:

In [33]:
dat = transformed_data.dropna()

In [51]:
data = data.dropna()

In [52]:
len(data)

5239

In [ ]:
!pip install ctgan

In [49]:
!git clone https://github.com/DAI-Lab/ctgan

Cloning into 'ctgan'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 275 (delta 111), reused 237 (delta 84), pack-reused 0
Receiving objects: 100% (275/275), 6.72 MiB | 9.58 MiB/s, done.
Resolving deltas: 100% (111/111), done.


In [53]:
from ctgan.cli import CTGANSynthesizer

In [54]:
ctgan_model = CTGANSynthesizer()

In [56]:
data.drop(columns='Unnamed: 0.1', inplace=True)

In [58]:
data.program_code.unique()

array(['prog_7', 'prog_1', 'prog_11', 'prog_5', 'prog_3', 'prog_9',
       'prog_10', 'prog_8', 'prog_13', 'prog_6', 'prog_2', 'prog_17',
       'prog_16', 'prog_4', 'prog_19', 'prog_15', 'prog_14', 'prog_12',
       'prog_18'], dtype=object)

In [67]:
data.math.min()

26.0

In [70]:
data.ues_calc_method.unique()

array(['doubling', 'summation'], dtype=object)

In [57]:
data.dtypes

program_code           object
ues                   float64
extra_points          float64
final_status           object
financing_source       object
re_enrolling             bool
foreign_lang          float64
hun_lang_lit          float64
math                  float64
history               float64
elective_subj         float64
foreign_lang_hs       float64
hun_lit_hs            float64
hun_lang_hs           float64
math_hs               float64
history_hs            float64
science_hs            float64
foreign_lang_score    float64
gender                 object
years_between         float64
surplus_score         float64
place_of_hs            object
ues_calc_method        object
dtype: object

In [73]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
print(scaler.fit(data[['surplus_score']]))

print(scaler.transform(data[['surplus_score']]))

MinMaxScaler(copy=True, feature_range=(0, 1))
[[0.01113019]
 [0.50441063]
 [0.1989225 ]
 ...
 [0.23918063]
 [0.00828844]
 [0.3125925 ]]


In [74]:
data['surplus_score'] = scaler.transform(data[['surplus_score']])

In [75]:
data.surplus_score.max()

1.0

In [76]:
data.to_csv('to_simulate.dat', sep=' ', index=False, header=False)

In [20]:
data_3.drop(columns=['Szervezet_neve', 'ks_helye']).to_csv('no_missing_space_delimited.dat', sep=' ', index=False, header=False)

In [30]:
data_3.drop(columns=['Szervezet_neve', 'ks_helye']).columns

Index(['Képzés', 'felvi_osszes_skalazott', 'Hozott/Tanulmányi_pontszám',
       'Szerzett/Érettségi_pontsz', 'tobblet_skalazott', 'Statusz_vegzett',
       'allamilag_tamogatott', 're_enrolled', 'idegen_nyelv',
       'magyar nyelv és irodalom', 'matematika', 'történelem',
       'választott_tárgy', 'idegen_nyelv_ks', 'magyar_irodalom_ks',
       'magyar_nyelvtan_ks', 'matematika_ks', 'történelem_ks',
       'választható_tantárgy_ks', 'nyv_pontok', 'nem_ferfi',
       'erettsegi_ota_eltelt_évek'],
      dtype='object')

In [77]:
ctgan_model = CTGANSynthesizer()

In [ ]:
!cd ctgan
!python -m ctgan.cli --data ./to_simulate.dat --meta  ./metadata.meta --max_epoch 2000 --model_dir ./output_dir/model --output generated_data --sample 50000

/home/nagy/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/home/nagy/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/home/nagy/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/home/nagy/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 